In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model


In [3]:
import pandas as pd
data=pd.read_csv("C:/Users/EBIN BABU/Downloads/archive (8)/combine.csv")

In [4]:
data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,55054,109.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,55055,52.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,46236,34.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,54863,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [5]:
data.shape

(2214469, 79)

In [6]:
data.dtypes

 Destination Port               object
 Flow Duration                 float64
 Total Fwd Packets             float64
 Total Backward Packets        float64
Total Length of Fwd Packets    float64
                                ...   
Idle Mean                      float64
 Idle Std                      float64
 Idle Max                      float64
 Idle Min                      float64
 Label                          object
Length: 79, dtype: object

In [7]:
data.duplicated().sum()

np.int64(271280)

In [8]:
data.drop_duplicates(inplace=True)
data.duplicated().sum()

np.int64(0)

In [9]:
data.isnull().sum()

 Destination Port              0
 Flow Duration                 1
 Total Fwd Packets             1
 Total Backward Packets        1
Total Length of Fwd Packets    1
                              ..
Idle Mean                      1
 Idle Std                      1
 Idle Max                      1
 Idle Min                      1
 Label                         1
Length: 79, dtype: int64

In [10]:
data.columns = data.columns.str.strip()

In [11]:

data['Total Fwd Packets'].fillna(data['Total Fwd Packets'].mode()[0], inplace=True)
data['Flow Duration'].fillna(data['Flow Duration'].mean(), inplace=True)
data['Total Backward Packets'].fillna(data['Total Backward Packets'].median(), inplace=True)
data['Total Length of Fwd Packets'].fillna(data['Total Length of Fwd Packets'].median(), inplace=True)
data['Idle Mean'].fillna(data['Idle Mean'].median(), inplace=True)
data['Idle Std'].fillna(data['Idle Std'].median(), inplace=True)
data['Idle Max'].fillna(data['Idle Max'].median(), inplace=True)
data['Idle Min'].fillna(data['Idle Min'].median(), inplace=True)
data.isnull().sum()


Destination Port               0
Flow Duration                  0
Total Fwd Packets              0
Total Backward Packets         0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
Idle Std                       0
Idle Max                       0
Idle Min                       0
Label                          1
Length: 79, dtype: int64

In [12]:

# 1. Remove the Label Column

y = data['Label']
data = data.drop(columns=['Label'])

# 2. Select Best Features

selected_features = [ 'Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
    'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Active Mean', 'Active Std', 'Idle Mean',
    'Idle Max','Idle Min']
X = data[selected_features]

# 3. Create New Features
#help the model understand hidden patterns
X['Packet_Ratio'] = X['Total Fwd Packets'] / (X['Total Backward Packets'] + 1)
#Reduce noise and make patterns clearer
X['Length_Ratio'] = X['Total Length of Fwd Packets'] / (X['Total Length of Bwd Packets'] + 1)
# Improve model accuracy
X['Flow_Bytes'] = X['Total Length of Fwd Packets'] + X['Total Length of Bwd Packets']
#Create useful relationships between columns
X['Fwd_Packet_Range'] = X['Fwd Packet Length Max'] - X['Fwd Packet Length Min']

# 4. Final Feature Set

print("Final Features Used for Model:")
print(X.head())


Final Features Used for Model:
  Destination Port  Flow Duration  Total Fwd Packets  Total Backward Packets  \
0            54865            3.0                2.0                     0.0   
1            55054          109.0                1.0                     1.0   
2            55055           52.0                1.0                     1.0   
3            46236           34.0                1.0                     1.0   
4            54863            3.0                2.0                     0.0   

   Total Length of Fwd Packets  Total Length of Bwd Packets  \
0                         12.0                          0.0   
1                          6.0                          6.0   
2                          6.0                          6.0   
3                          6.0                          6.0   
4                         12.0                          0.0   

   Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  \
0                    6.0          

In [13]:
# Remove non-numeric hidden characters from the dataset
X = X.replace(r'[^\d\.]', '', regex=True)
X = X.apply(pd.to_numeric, errors='coerce')



#train test split
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)




In [14]:

# Define input_dim as the number of features
input_dim = X_train_scaled.shape[1]

# Build Autoencoder Model
input_layer = Input(shape=(input_dim,))
encoder = Dense(16, activation="relu")(input_layer)
encoder = Dense(8, activation="relu")(encoder)
encoder = Dense(4, activation="relu")(encoder)

decoder = Dense(8, activation="relu")(encoder)
decoder = Dense(16, activation="relu")(decoder)
output_layer = Dense(input_dim, activation="linear")(decoder)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer="adam", loss="mse")

autoencoder.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 19)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 19)             │           323 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 999 (3.90 KB)

 Trainable params: 999 (3.90 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:

#Train Autoencoder
history = autoencoder.fit(
    X_train_scaled, X_train_scaled,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

Epoch 1/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 96s 2ms/step - loss: nan - val_loss: nan
Epoch 2/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 54s 1ms/step - loss: nan - val_loss: nan
Epoch 3/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 54s 1ms/step - loss: nan - val_loss: nan
Epoch 4/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 51s 1ms/step - loss: nan - val_loss: nan
Epoch 5/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 47s 1ms/step - loss: nan - val_loss: nan
Epoch 6/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 47s 1ms/step - loss: nan - val_loss: nan
Epoch 7/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 47s 1ms/step - loss: nan - val_loss: nan
Epoch 8/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 165s 4ms/step - loss: nan - val_loss: nan
Epoch 9/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 206s 5ms/step - loss: nan - val_loss: nan
Epoch 10/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 474s 11ms/step - loss: nan - val_loss: nan
Epoch 11/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 208s 5ms/step - loss: nan - val_loss: nan
Epoch 12/50
43722/43722 ━━━━━━━━━━━━━━━━━━━━ 210s 5ms/step

In [17]:

# Predict reconstructed values
import numpy as np
X_train_pred = autoencoder.predict(X_train_scaled)
train_errors = np.mean(np.square(X_train_scaled - X_train_pred), axis=1)

X_test_pred = autoencoder.predict(X_test_scaled)
test_errors = np.mean(np.square(X_test_scaled - X_test_pred), axis=1)



48580/48580 ━━━━━━━━━━━━━━━━━━━━ 119s 2ms/step 
12145/12145 ━━━━━━━━━━━━━━━━━━━━ 8s 668us/step


In [18]:
#Set Anomaly Threshold
threshold = np.percentile(train_errors, 95)
print("Threshold:", threshold)


Threshold: nan


In [19]:
#Predict Anomalies
y_pred = [1 if e > threshold else 0 for e in test_errors]


In [26]:
# Evaluation (If labels exist)

if 'Label' in data.columns:
    print("Label column found. Performing evaluation...")

    # Convert labels to binary if needed
    # Assuming: Benign = 0, Attack = 1
    y_true = data.loc[X_test.index, 'Label']

    # If labels are text, convert them
    if y_true.dtype == 'object':
        y_true = y_true.apply(lambda x: 0 if x.lower() in ['benign', 'normal'] else 1)

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))



In [28]:
#Save Model for Deployment
import joblib
autoencoder.save("autoencoder_model.h5")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(threshold, "threshold.pkl")


['threshold.pkl']

In [32]:
#Create Prediction Function
import numpy as np
import joblib
from tensorflow.keras.models import load_model

# Load saved artifacts
model = load_model("autoencoder_model.h5", compile=False)
scaler = joblib.load("scaler.pkl")
threshold = joblib.load("threshold.pkl")

autoencoder.save("autoencoder_model.keras")

model = load_model("autoencoder_model.keras")

def detect_anomaly(feature_vector):
    """
    feature_vector: array/list of ALL features
    in the SAME ORDER used during training
    """
    data = np.array([feature_vector])
    data_scaled = scaler.transform(data)

    reconstructed = model.predict(data_scaled, verbose=0)
    error = np.mean(np.square(data_scaled - reconstructed))

    return "Anomaly Detected" if error > threshold else "Normal Traffic"


# Example input
print(detect_anomaly(300, 1.2, 1200, 800))


In [ ]:
| packet_size | duration | src_bytes | dst_bytes |
| ----------- | -------- | --------- | --------- |
| 350         | 2.1      | 1400      | 900       |


In [40]:
def detect_anomaly_from_partial_input(packet_size, duration, src_bytes, dst_bytes):
    # Create zero-filled feature vector
    feature_vector = np.zeros(X.shape[1])

    feature_names = X.columns.tolist()

    feature_map = {
        'Fwd Packet Length Mean': packet_size,
        'Flow Duration': duration,
        'Total Length of Fwd Packets': src_bytes,
        'Total Length of Bwd Packets': dst_bytes
    }

    for i, col in enumerate(feature_names):
        if col in feature_map:
            feature_vector[i] = feature_map[col]

    return detect_anomaly(feature_vector)


In [41]:
print(detect_anomaly_from_partial_input(
    packet_size=350,
    duration=2.1,
    src_bytes=1400,
    dst_bytes=900
))


Normal Traffic
